<a href="https://colab.research.google.com/github/tb-harris/neuroscience-2024/blob/main/Feature_Extractor_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# A few tools for downloading the data
from io import StringIO
import requests
!pip install neuron_morphology
from neuron_morphology.swc_io import morphology_from_swc
from neuron_morphology.feature_extractor.data import Data

In [ ]:
import tempfile
def data_from_url(morphology_url):
    response = requests.get(morphology_url)
    morphology_swc = response.text

    with tempfile.NamedTemporaryFile(delete=False, mode='w') as temp_file:
        temp_file.write(morphology_swc)
        temp_file_path = temp_file.name

    return Data(morphology_from_swc(temp_file_path))

In [ ]:
# fetch a published reconstruction
IVSCC_swc_file = "http://celltypes.brain-map.org/api/v2/well_known_file_download/491120375"
more_test_data = data_from_url("http://celltypes.brain-map.org/api/v2/well_known_file_download/692297222")
test_data = data_from_url(IVSCC_swc_file)

In [ ]:
from neuron_morphology.feature_extractor.feature_extractor import FeatureExtractor
from neuron_morphology.features.dimension import dimension
from neuron_morphology.features.intrinsic import num_branches, num_tips, mean_fragmentation
from neuron_morphology.features.size import total_length
from neuron_morphology.features.default_features import default_features
from neuron_morphology.constants import AXON, BASAL_DENDRITE, SOMA, APICAL_DENDRITE
# a utility for flattening outputs
from neuron_morphology.feature_extractor.utilities import unnest



features = [
    dimension,
    num_branches,
    num_tips,
    # mean_fragmentation
]

# features = default_features


# make a pipeline
results = (
    FeatureExtractor()
    .register_features(features)
    .extract(test_data)
    # .extract(more_test_data)
    .results
)


unnest(results)


# dendrite_node_types = [BASAL_DENDRITE, APICAL_DENDRITE]
# total_length(test_data, dendrite_node_types)

In [ ]:
NEURITE_SPECIALIZATIONS

In [ ]:
from neuron_morphology.feature_extractor.feature_specialization import NEURITE_SPECIALIZATIONS, AxonSpec, BasalDendriteSpec
from neuron_morphology.feature_extractor.marked_feature import specialize

In [ ]:
axon_num_branches = specialize(num_branches, {AxonSpec}) # note that this is a set!
unnest(
    FeatureExtractor()
    .register_features([axon_num_branches])
    .extract(test_data)
    .results
)

In [ ]:
across_neurites = [specialize(num_branches, NEURITE_SPECIALIZATIONS)]

# this syntax is (almost) equivalent to the above pipeline, but we keep the extractor around
extractor = FeatureExtractor()
extractor.register_features(across_neurites)
run = extractor.extract(test_data)
across_neurite_results = run.results

unnest(across_neurite_results)

In [ ]:
# ... by keeping the extractor we can re-use it on new data

run_two = extractor.extract(more_test_data)
unnest(run_two.results)

In [ ]:
# Import all necessary packages and libraries
!pip install allensdk
!pip install pandas==1.5.3
import pandas as pd
!pip install neuron_morphology
import sys
sys.path.insert(0, "../")
#import warnings
#warnings.filterwarnings('ignore')

from io import StringIO
import requests
from neuron_morphology.swc_io import morphology_from_swc
from neuron_morphology.feature_extractor.data import Data
from neuron_morphology.feature_extractor.feature_extractor import FeatureExtractor
import neuron_morphology.feature_extractor.feature_writer as fw
from neuron_morphology.features.default_features import default_features

import matplotlib.pyplot as plt
import json

#Mount google drive
from google.colab import drive
drive.mount('/content/drive')
#metadata
path = '/content/drive/MyDrive/PatchSeq/20200711_patchseq_metadata_mouse - 20200711_patchseq_metadata_mouse.csv'
metadata = pd.read_csv(path)
# manifest
path1 = '/content/drive/MyDrive/PatchSeq/2021-09-13_mouse_file_manifest - 2021-09-13_mouse_file_manifest.csv'
manifest = pd.read_csv(path1)

#get 573 morph file and their ids
swc_urls = manifest.loc[
    (manifest["file_type"] == "transformed_swc")
]

# select the "cell_specimen_id" column of the swc_urls DataFrame
selected_ids = swc_urls["cell_specimen_id"].tolist()

archive_url = swc_urls["archive_uri"].values

#5m 10s
for url in archive_url:
  !wget {url}


#get morph files
swc_path = swc_urls["file_name"].values

import os
# convert relative file paths to absolute file paths
swc_paths = [os.path.abspath(path) for path in swc_urls["file_name"].values.tolist()]

morphologies = []
# pass the file paths to morphology_from_swc() function
for path in swc_paths:
  morph = morphology_from_swc(path)
  morphologies.append(morph)

In [ ]:
#this utilizes the feature extractor library to get specific information about the neurons
#in this case we are extracting information regarding the total dendrite length of neurons
from neuron_morphology.features.size import total_length
from neuron_morphology.features.intrinsic import mean_fragmentation
from neuron_morphology.feature_extractor.feature_specialization import NEURITE_SPECIALIZATIONS
from neuron_morphology.feature_extractor.marked_feature import specialize

from neuron_morphology.feature_extractor.feature_extractor import FeatureExtractor

fe = FeatureExtractor()
mean_fragmentation = [specialize(mean_fragmentation, NEURITE_SPECIALIZATIONS)]
fe.register_features(mean_fragmentation)
#------------
from neuron_morphology.feature_extractor.data import Data

counter = 0

# for morphology in morphologies:
#     print(counter)
#     data = Data(morphology)
#     feature_extraction_run = fe.extract(data)
#     results = run.results
#     unnest(results)
#     counter += 1

morph = morphologies[6]
data = Data(morph)
feature_extraction_run = fe.extract(data)
results = feature_extraction_run.results
unnest(results)


In [ ]:
across_neurites = [specialize(num_branches, NEURITE_SPECIALIZATIONS)]

# this syntax is (almost) equivalent to the above pipeline, but we keep the extractor around
extractor = FeatureExtractor()
extractor.register_features(across_neurites)
morph = morphologies[5]
data = Data(morph)
run = extractor.extract(data)
across_neurite_results = run.results

unnest(across_neurite_results)